In [1]:
from helpers import cd_to_datetime, datetime_to_str

class NearEarthObject:
    """A near-Earth object (NEO).

    An NEO encapsulates semantic and physical parameters about the object, such
    as its primary designation (required, unique), IAU name (optional), diameter
    in kilometers (optional - sometimes unknown), and whether it's marked as
    potentially hazardous to Earth.

    A `NearEarthObject` also maintains a collection of its close approaches -
    initialized to an empty collection, but eventually populated in the
    `NEODatabase` constructor.
    """

    def __init__(self, designation, name=None, diameter=float('nan'), hazardous='N'):
        """Create a new `NearEarthObject`.
        
        :param info: 
        required unique designation (str), 
        optional name (str), diameter (float), hazardous (bool)
        """
        
        self.designation = str(designation)
        self.name = str(name).title()
        self.diameter = diameter
        self.hazardous = hazardous
        
        self.approaches = []

    @property
    def fullname(self):
        """Return a representation of the full name of this NEO."""
        
        if self.name is None:
            return self.designation      
        else:
            return self.designation + " (" + self.name + ")"
    
    @property
    def hazard(self):
        """Return 'is/is not' depending on boolean value of hazardous."""
        
        if self.hazardous == 'N':
            hazard = 'is not'
        else:
            hazard = 'is'
        return hazard

    def __str__(self):
        """Return `str(self)`."""
        
        formatted_string = "NEO {} has a diameter {} km and {} potentially hazardous.".format(
        self.fullname, self.diameter, self.hazard
        )

        return formatted_string
  
    
    def __hash__(self):
        return hash(str(self))
    
    def __eq__(self, neo):
        return self.designation is neo.designation

class CloseApproach:
    """A close approach to Earth by an NEO.

    A `CloseApproach` encapsulates information about the NEO's close approach to
    Earth, such as the date and time (in UTC) of closest approach, the nominal
    approach distance in astronomical units, and the relative approach velocity
    in kilometers per second.

    A `CloseApproach` also maintains a reference to its `NearEarthObject` -
    initally, this information (the NEO's primary designation) is saved in a
    private attribute, but the referenced NEO is eventually replaced in the
    `NEODatabase` constructor.
    """

    def __init__(self, des, time, distance=float('nan'), velocity=float('nan'), neo=None):
        """Create a new `CloseApproach`.

        :param info: 
        required unique designation (str), can be passed from the neo type'
        required date (str);
        optional: distance (float), velocity (bool), neo (neo).
        """

        
        self._designation = des
        self.time = cd_to_datetime(time)        
        self.distance = distance
        self.velocity = velocity
        self.neo = neo

        
    @property
    def time_str(self):
        """Return a formatted representation of this `CloseApproach`'s approach time.

        The value in `self.time` should be a Python `datetime` object. While a
        `datetime` object has a string representation, the default representation
        includes seconds - significant figures that don't exist in our input
        data set.

        The `datetime_to_str` method converts a `datetime` object to a
        formatted string that can be used in human-readable representations and
        in serialization to CSV and JSON files.
        """

        return datetime_to_str(self.time)

    
    def __str__(self):
        """Return `str(self)`."""
        
        formatted_string = "- On {} NEO {} approaching with speed {} km/s and distance {} au.".format(
        self.time_str, self.neo.fullname, self.velocity, self.distance
        )
            
        return formatted_string

    def __repr__(self):
        """Return `repr(self)`, a computer-readable string representation of this object."""
        
        formatted_string = "CloseApproach(time={}, distance={}, velocity={}, neo={})".format(
        self.time_str, self.distance, self.velocity, self.neo)
        
        return formatted_string

In [2]:
n = NearEarthObject(123, 'Barsik', 21.21, True)
cap1 = CloseApproach(123, '1900-Dec-27 12:12', 10, 0.1, n)
cap2 = CloseApproach(123, '1910-Dec-27 12:12', 11, 1.1, n)

In [3]:
print(n)

NEO 123 (Barsik) has a diameter 21.21 km and is potentially hazardous.


In [4]:
print(cap1, cap2)

- On 1900-12-27 12:12 NEO 123 (Barsik) approaching with speed 0.1 km/s and distance 10 au. - On 1910-12-27 12:12 NEO 123 (Barsik) approaching with speed 1.1 km/s and distance 11 au.


In [5]:
n.approaches

[]

In [6]:
n.approaches.append(cap1)
n.approaches.append(cap2)

In [7]:
n.approaches

[CloseApproach(time=1900-12-27 12:12, distance=10, velocity=0.1, neo=NEO 123 (Barsik) has a diameter 21.21 km and is potentially hazardous.),
 CloseApproach(time=1910-12-27 12:12, distance=11, velocity=1.1, neo=NEO 123 (Barsik) has a diameter 21.21 km and is potentially hazardous.)]

### Loading datasets with `extract.py`

In [8]:
"""Extract data on near-Earth objects and close approaches from CSV and JSON files.

The `load_neos` function extracts NEO data from a CSV file, formatted as
described in the project instructions, into a collection of `NearEarthObject`s.

The `load_approaches` function extracts close approach data from a JSON file,
formatted as described in the project instructions, into a collection of
`CloseApproach` objects.

The main module calls these functions with the arguments provided at the command
line, and uses the resulting collections to build an `NEODatabase`.

You'll edit this file in Task 2.
"""
import csv
import json
from collections import defaultdict

#from models import NearEarthObject, CloseApproach


def load_neos(neo_csv_path = 'data/neos.csv') -> dict:
    """Read near-Earth object information from a CSV file.

    :param neo_csv_path: A path to a CSV file containing data about near-Earth objects.
    :return: A collection of `NearEarthObject`s.
    """

    with open(neo_csv_path, 'r') as file:
        neo_data = [row for row in csv.DictReader(file)]
        
    neos_dict = {}
    for i in range(len(neo_data)):
        neos_dict[neo_data[i]['pdes']] = NearEarthObject(
            neo_data[i]['pdes'], 
            neo_data[i]['name'], 
            neo_data[i]['diameter'], 
            neo_data[i]['pha']
        )
    
    return neos_dict



def load_approaches(cad_json_path = 'data/cad.json') -> dict:
    """Read close approach data from a JSON file.

    :param neo_csv_path: A path to a JSON file containing data about close approaches.
    :return: A collection of `CloseApproach`es.
    """

    with open(cad_json_path, 'r') as file:
        cads_data = sorted(json.load(file)['data'])

    cads_dict = {}

    for i in range(len(cads_data)):
        des = cads_data[i][0]
        time = cads_data[i][3]
        distance = cads_data[i][5] 
        velocity = cads_data[i][7]

        if des not in cads_dict:
            cads_dict[des] = []
            cads_dict[des].append({time : CloseApproach(des, time, distance, velocity)})
        
        else:
            cads_dict[des].append({time : CloseApproach(des, time, distance, velocity)})

    return cads_dict


In [12]:
neos_dict = load_neos()
cads_dict = load_approaches()

In [88]:
"""A database encapsulating collections of near-Earth objects and their close approaches.
    
    A `NEODatabase` holds an interconnected data set of NEOs and close approaches.
    It provides methods to fetch an NEO by primary designation or by name, as well
    as a method to query the set of close approaches that match a collection of
    user-specified criteria.
    
    Under normal circumstances, the main module creates one NEODatabase from the
    data on NEOs and close approaches extracted by `extract.load_neos` and
    `extract.load_approaches`.
    
    """
#from extract import load_neos, load_approaches

class NEODatabase:
    """A database of near-Earth objects and their close approaches.
        
        A `NEODatabase` contains a collection of NEOs and a collection of close
        approaches. It additionally maintains a few auxiliary data structures to
        help fetch NEOs by primary designation or by name and to help speed up
        querying for close approaches that match criteria.
        """
    def __init__(self, neos_dict, approaches_dict):
        """Create a new `NEODatabase`.
            
            As a precondition, this constructor assumes that the collections of NEOs
            and close approaches haven't yet been linked - that is, the
            `.approaches` attribute of each `NearEarthObject` resolves to an empty
            collection, and the `.neo` attribute of each `CloseApproach` is None.
            
            However, each `CloseApproach` has an attribute (`._designation`) that
            matches the `.designation` attribute of the corresponding NEO. This
            constructor modifies the supplied NEOs and close approaches to link them
            together - after it's done, the `.approaches` attribute of each NEO has
            a collection of that NEO's close approaches, and the `.neo` attribute of
            each close approach references the appropriate NEO.
            
            :param neos: A collection of `NearEarthObject`s.
            :param approaches: A collection of `CloseApproach`es.
            """
        self.neos_dict = neos_dict
        self.approaches_dict = approaches_dict


        for pdes, neo in self.neos_dict.items():
            for des, cads in self.approaches_dict.items():
                if pdes == des:
                    for i in range(len(cads)):
                        for value in cads[i].values():
                            self.neos_dict[pdes].approaches.append(value)
                            value.neo = neo
    


    def get_neo_by_designation(self, designation):
        """Find and return an NEO by its primary designation.

            If no match is found, return `None` instead.

            Each NEO in the data set has a unique primary designation, as a string.

            The matching is exact - check for spelling and capitalization if no
            match is found.

            :param designation: The primary designation of the NEO to search for.
            :return: The `NearEarthObject` with the desired primary designation, or `None`.
            """
        if designation in self.neos_dict.keys():
            print(self.neos_dict[designation])
        else:
            print("No NEO designation {} was found.".format(designation))
        return
                    


    def get_neo_by_name(self, name):
        """Find and return an NEO by its name.

            If no match is found, return `None` instead.

            Not every NEO in the data set has a name. No NEOs are associated with
            the empty string nor with the `None` singleton.

            The matching is exact - check for spelling and capitalization if no
            match is found.

            :param name: The name, as a string, of the NEO to search for.
            :return: The `NearEarthObject` with the desired name, or `None`.
            """
        for neo in self.neos_dict.values():
            if name == neo.name:
                return neo
                break 
            else:
                print("No NEO name {} was found.".format(name))
                break
        return

    def query(self, filters=()):
        """Query close approaches to generate those that match a collection of filters.

            This generates a stream of `CloseApproach` objects that match all of the
            provided filters.

            If no arguments are provided, generate all known close approaches.

            The `CloseApproach` objects are generated in internal order, which isn't
            guaranteed to be sorted meaninfully, although is often sorted by time.

            :param filters: A collection of filters capturing user-specified criteria.
            :return: A stream of matching `CloseApproach` objects.
            """

            # TODO: Generate `CloseApproach` objects that match all of the filters.
        for approach in self.approaches_dict:
            mapped = map(lambda x: x(approach), filters)
            if all(flag is True for flag in mapped):
                yield approach


In [89]:
neo_db = NEODatabase(neos_dict, cads_dict)

In [94]:
neo_db.get_neo_by_designation('433')
neo_db.get_neo_by_name('Eros').approaches

NEO 433 (Eros) has a diameter 16.84 km and is not potentially hazardous.


[CloseApproach(time=1900-12-27 01:30, distance=0.314928770649129, velocity=5.57862036137798, neo=NEO 433 (Eros) has a diameter 16.84 km and is not potentially hazardous.),
 CloseApproach(time=1907-11-05 03:31, distance=0.471485325584323, velocity=4.39449034015508, neo=NEO 433 (Eros) has a diameter 16.84 km and is not potentially hazardous.),
 CloseApproach(time=1917-04-20 21:19, distance=0.499257057462409, velocity=4.81678458316768, neo=NEO 433 (Eros) has a diameter 16.84 km and is not potentially hazardous.),
 CloseApproach(time=1924-03-05 22:13, distance=0.35978626105059, velocity=4.5960536046792, neo=NEO 433 (Eros) has a diameter 16.84 km and is not potentially hazardous.),
 CloseApproach(time=1931-01-30 04:07, distance=0.174072995458928, velocity=5.92081851998368, neo=NEO 433 (Eros) has a diameter 16.84 km and is not potentially hazardous.),
 CloseApproach(time=1938-01-13 22:04, distance=0.215004986486131, velocity=6.08405229549247, neo=NEO 433 (Eros) has a diameter 16.84 km and is

In [74]:
neo_db.get_neo_by_designation('433')

NEO 433 (Eros) has a diameter 16.84 km and is not potentially hazardous.


In [73]:
print(neos_dict['433']) #.name

NEO 433 (Eros) has a diameter 16.84 km and is not potentially hazardous.


In [78]:
neo_db.get_neo_by_name('w')


No NEO name w was found.


In [31]:
print('Print all cads for a neo', neo_db.get_neo_by_designation('433').name)

NEO 433 (Eros) has a diameter 16.84 km and is not potentially hazardous.


AttributeError: 'NoneType' object has no attribute 'name'

In [26]:
neo_db.get_neo_by_designation('433')

NEO 433 (Eros) has a diameter 16.84 km and is not potentially hazardous.


In [386]:
neo_db.get_neo_by_designation('0')

No NEO designation 0 was found.


In [387]:
neo_db.get_neo_by_name('Eros').designation

'433'

In [388]:
neo_db.get_neo_by_name('Pooo')

No NEO name Pooo was found.


In [393]:
for i in range(len(neo_db.approaches_dict['100004'])):
    for value in neo_db.approaches_dict['1036'][i].values():
        print(value.neo)

NEO 1036 (Ganymed) has a diameter 37.675 km and is not potentially hazardous.
NEO 1036 (Ganymed) has a diameter 37.675 km and is not potentially hazardous.
NEO 1036 (Ganymed) has a diameter 37.675 km and is not potentially hazardous.
NEO 1036 (Ganymed) has a diameter 37.675 km and is not potentially hazardous.
NEO 1036 (Ganymed) has a diameter 37.675 km and is not potentially hazardous.
NEO 1036 (Ganymed) has a diameter 37.675 km and is not potentially hazardous.


IndexError: list index out of range

### Filters functionality

In [ ]:
"""Provide filters for querying close approaches and limit the generated results.

The `create_filters` function produces a collection of objects that is used by
the `query` method to generate a stream of `CloseApproach` objects that match
all of the desired criteria. The arguments to `create_filters` are provided by
the main module and originate from the user's command-line options.

This function can be thought to return a collection of instances of subclasses
of `AttributeFilter` - a 1-argument callable (on a `CloseApproach`) constructed
from a comparator (from the `operator` module), a reference value, and a class
method `get` that subclasses can override to fetch an attribute of interest from
the supplied `CloseApproach`.

The `limit` function simply limits the maximum number of values produced by an
iterator.

You'll edit this file in Tasks 3a and 3c.
"""
import operator


class UnsupportedCriterionError(NotImplementedError):
    """A filter criterion is unsupported."""


class AttributeFilter:
    """A general superclass for filters on comparable attributes.

    An `AttributeFilter` represents the search criteria pattern comparing some
    attribute of a close approach (or its attached NEO) to a reference value. It
    essentially functions as a callable predicate for whether a `CloseApproach`
    object satisfies the encoded criterion.

    It is constructed with a comparator operator and a reference value, and
    calling the filter (with __call__) executes `get(approach) OP value` (in
    infix notation).

    Concrete subclasses can override the `get` classmethod to provide custom
    behavior to fetch a desired attribute from the given `CloseApproach`.
    """
    def __init__(self, op, value):
        """Construct a new `AttributeFilter` from an binary predicate and a reference value.

        The reference value will be supplied as the second (right-hand side)
        argument to the operator function. For example, an `AttributeFilter`
        with `op=operator.le` and `value=10` will, when called on an approach,
        evaluate `some_attribute <= 10`.

        :param op: A 2-argument predicate comparator (such as `operator.le`).
        :param value: The reference value to compare against.
        """
        self.op = op
        self.value = value

    def __call__(self, approach):
        """Invoke `self(approach)`."""
        return self.op(self.get(approach), self.value)

    @classmethod
    def get(cls, approach):
        """Get an attribute of interest from a close approach.

        Concrete subclasses must override this method to get an attribute of
        interest from the supplied `CloseApproach`.

        :param approach: A `CloseApproach` on which to evaluate this filter.
        :return: The value of an attribute of interest, comparable to `self.value` via `self.op`.
        """
        raise UnsupportedCriterionError

    def __repr__(self):
        return f"{self.__class__.__name__}(op=operator.{self.op.__name__}, value={self.value})"


def create_filters(date=None, start_date=None, end_date=None,
                   distance_min=None, distance_max=None,
                   velocity_min=None, velocity_max=None,
                   diameter_min=None, diameter_max=None,
                   hazardous=None):
    """Create a collection of filters from user-specified criteria.

    Each of these arguments is provided by the main module with a value from the
    user's options at the command line. Each one corresponds to a different type
    of filter. For example, the `--date` option corresponds to the `date`
    argument, and represents a filter that selects close approaches that occured
    on exactly that given date. Similarly, the `--min-distance` option
    corresponds to the `distance_min` argument, and represents a filter that
    selects close approaches whose nominal approach distance is at least that
    far away from Earth. Each option is `None` if not specified at the command
    line (in particular, this means that the `--not-hazardous` flag results in
    `hazardous=False`, not to be confused with `hazardous=None`).

    The return value must be compatible with the `query` method of `NEODatabase`
    because the main module directly passes this result to that method. For now,
    this can be thought of as a collection of `AttributeFilter`s.

    :param date: A `date` on which a matching `CloseApproach` occurs.
    :param start_date: A `date` on or after which a matching `CloseApproach` occurs.
    :param end_date: A `date` on or before which a matching `CloseApproach` occurs.
    :param distance_min: A minimum nominal approach distance for a matching `CloseApproach`.
    :param distance_max: A maximum nominal approach distance for a matching `CloseApproach`.
    :param velocity_min: A minimum relative approach velocity for a matching `CloseApproach`.
    :param velocity_max: A maximum relative approach velocity for a matching `CloseApproach`.
    :param diameter_min: A minimum diameter of the NEO of a matching `CloseApproach`.
    :param diameter_max: A maximum diameter of the NEO of a matching `CloseApproach`.
    :param hazardous: Whether the NEO of a matching `CloseApproach` is potentially hazardous.
    :return: A collection of filters for use with `query`.
    """
    # TODO: Decide how you will represent your filters.
    return ()


def limit(iterator, n=None):
    """Produce a limited stream of values from an iterator.

    If `n` is 0 or None, don't limit the iterator at all.

    :param iterator: An iterator of values.
    :param n: The maximum number of values to produce.
    :yield: The first (at most) `n` values from the iterator.
    """
    # TODO: Produce at most `n` values from the given iterator.
    return iterator